In [2]:
import os, glob
import pandas as pd
from io import StringIO
import pyBigWig
import subprocess
import pybedtools 
pybedtools.helpers.set_tempdir('/data/projects/temp')

In [3]:
enhancer_file_path = "/data/projects/Enhancer/RECOMB_2024/Enhancer_BED.tsv"

In [4]:
df_enhancer = pd.read_csv(enhancer_file_path, sep="\t")
df_enhancer

,chromosome_name,enhancer_start,enhancer_end
0,chr6,167157937,167158136
1,chr8,140206853,140207052
2,chr5,170638695,170638894
3,chr1,63043547,63043746
4,chr19,19238266,19238465
...,...,...,...
7774,chr1,204552002,204552201
7775,chr9,38725772,38725971
7776,chr7,27950006,27950205
7777,chr2,171047833,171048032


## For Core Promoter Region

In [5]:
df_CoreProm = pd.read_csv("/data/private/pdutta/DNABERT_data/Core_promoters/Core_Prom_bed.tsv", sep="\t")

In [6]:
df_CoreProm

,Chr,gene_id,transcript_id,exon_id,strand,TSS,start,end
0,chr7,ENSG00000004059.11,ENST00000000233.10,ENSE00001872691.2,+,127588411,127588366,127588455
1,chr7,ENSG00000001626.16,ENST00000003084.11,ENSE00001343851.2,+,117480025,117479980,117480069
2,chr7,ENSG00000001630.17,ENST00000003100.13,ENSE00001216550.10,-,92134477,92134433,92134522
3,chr7,ENSG00000002933.9,ENST00000004103.8,ENSE00001827596.2,+,150800769,150800724,150800813
4,chr7,ENSG00000004799.8,ENST00000005178.6,ENSE00001133344.6,-,95596516,95596472,95596561
...,...,...,...,...,...,...,...,...
236970,chr15,ENSG00000103876.14,ENST00000684569.1,ENSE00003921718.1,+,80153010,80152965,80153054
236971,chr15,ENSG00000213614.11,ENST00000684602.1,ENSE00002620879.1,-,72375989,72375945,72376034
236972,chr15,ENSG00000213614.11,ENST00000684667.1,ENSE00002584031.1,-,72375972,72375928,72376017
236973,chr15,ENSG00000104133.16,ENST00000684676.1,ENSE00003922470.1,-,44663688,44663644,44663733


In [8]:
str_io_CoreProm = StringIO()
df_CoreProm[['Chr','start','end','transcript_id','strand']].to_csv(str_io_CoreProm, sep='\t', index=False, header=False)
str_io_CoreProm.seek(0)
CoreProm_bed = pybedtools.BedTool(str_io_CoreProm.read(), from_string=True)

In [9]:
str_io_enhancer = StringIO()
df_enhancer[['chromosome_name','enhancer_start','enhancer_end']].to_csv(str_io_enhancer, sep='\t', index=False, header=False)
str_io_enhancer.seek(0)
enhancer_bed = pybedtools.BedTool(str_io_enhancer.read(), from_string=True)

In [10]:
# intersect the files
intersect_bed = CoreProm_bed.intersect(enhancer_bed, wa=True, wb=True)

In [11]:
for i, interval in enumerate(intersect_bed):
    if i < 5:
        print(interval)
    else:
        break

chr7	6004341	6004430	ENST00000365120.1	-	chr7	6004337	6004536

chr7	77545707	77545796	ENST00000418110.5	+	chr7	77545645	77545844

chr7	42113320	42113409	ENST00000423237.1	+	chr7	42113334	42113533

chr7	18949718	18949807	ENST00000452663.1	-	chr7	18949599	18949798

chr7	73543849	73543938	ENST00000481667.1	-	chr7	73543761	73543960



In [12]:
intersect_df = intersect_bed.to_dataframe()

In [14]:
intersect_df

,chrom,start,end,name,score,strand,thickStart,thickEnd
0,chr7,6004341,6004430,ENST00000365120.1,-,chr7,6004337,6004536
1,chr7,77545707,77545796,ENST00000418110.5,+,chr7,77545645,77545844
2,chr7,42113320,42113409,ENST00000423237.1,+,chr7,42113334,42113533
3,chr7,18949718,18949807,ENST00000452663.1,-,chr7,18949599,18949798
4,chr7,73543849,73543938,ENST00000481667.1,-,chr7,73543761,73543960
...,...,...,...,...,...,...,...,...
143,chr18,10681684,10681773,ENST00000579151.1,-,chr18,10681696,10681895
144,chr15,36895258,36895347,ENST00000557992.1,-,chr15,36895234,36895433
145,chr15,44575228,44575317,ENST00000558080.1,-,chr15,44575245,44575444
146,chr15,36895243,36895332,ENST00000559972.1,-,chr15,36895234,36895433


In [15]:
intersect_df.to_csv("/data/projects/Enhancer/RECOMB_2024/Intersected_data/CoreProm_Enhancer_intersect.tsv", sep='\t', index=False)
#output_df.to_csv("/data/private/pdutta/DNABERT_data/Splice_sites/Acceptor_raw_files/PART_Acceptor_Transcript_intersect_details.tsv", sep='\t', index=False)

In [16]:
chr_wise_DBSNP_files = glob.glob('/data/projects/VCF_files/DBSNP/ChrWise_Data/*.csv')

In [17]:
for dbsnp_file in chr_wise_DBSNP_files:
    # Read the csv file into a DataFrame
    print(dbsnp_file)
    df = pd.read_csv(dbsnp_file, header=None, sep= ',', usecols=[0, 1, 2, 3, 4, 5,6])
    df = df.iloc[1:]
    print(df.head())
    
    # Convert the DataFrame to a BedTool
    dbsnp = pybedtools.BedTool.from_dataframe(df)
    
    # Intersect the BedTool with the transcript file
    intersected = intersect_bed.intersect(dbsnp, wa=True, wb=True)
    
    # Save the intersected BedTool to a bed file
    output_file = "/data/projects/Enhancer/RECOMB_2024/Intersected_data/CoreProm/Chrwise/intersected_DBSNP_transcript_" + dbsnp_file.split('/')[-1].replace('.csv', '.tsv')
    intersected.saveas(output_file)

/data/projects/VCF_files/DBSNP/ChrWise_Data/chr7_data.csv


/data/projects/temp/ipykernel_1148936/2691389721.py:4: DtypeWarning: Columns (1,2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(dbsnp_file, header=None, sep= ',', usecols=[0, 1, 2, 3, 4, 5,6])


      0      1      2             3  4  5   6
1  chr7  10001  10002  rs1583156258  T  1  C,
2  chr7  10002  10003  rs1583156262  A  1  G,
3  chr7  10003  10004  rs1583156265  A  1  C,
4  chr7  10004  10005  rs1583156268  C  1  A,
5  chr7  10006  10007  rs1781584841  C  1  T,
/data/projects/VCF_files/DBSNP/ChrWise_Data/chr13_data.csv


/data/projects/temp/ipykernel_1148936/2691389721.py:4: DtypeWarning: Columns (1,2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(dbsnp_file, header=None, sep= ',', usecols=[0, 1, 2, 3, 4, 5,6])


       0         1         2             3  4  5   6
1  chr13  16000025  16000026  rs1175737345  T  1  A,
2  chr13  16000063  16000064  rs1957610123  G  1  A,
3  chr13  16000079  16000080  rs1957610132  G  1  T,
4  chr13  16000081  16000082  rs1957610136  A  1  C,
5  chr13  16000100  16000101  rs1957610144  T  1  C,
/data/projects/VCF_files/DBSNP/ChrWise_Data/chrY_data.csv


...

      0      1      2             3   4  5       6
1  chr3  10008  10009  rs1339898020   A  3  C,G,T,
2  chr3  10009  10010  rs1695495101   A  1      C,
3  chr3  10010  10011  rs1695495136   C  1      A,
4  chr3  10014  10015  rs1206436362   A  1      C,
5  chr3  10014  10016  rs1695495167  AA  1    AAA,
/data/projects/VCF_files/DBSNP/ChrWise_Data/chr17_data.csv


/data/projects/temp/ipykernel_1148936/2691389721.py:4: DtypeWarning: Columns (1,2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(dbsnp_file, header=None, sep= ',', usecols=[0, 1, 2, 3, 4, 5,6])


       0      1      2             3  4  5   6
1  chr17  60054  60055  rs2039333033  G  1  A,
2  chr17  60086  60087  rs2039333047  C  1  T,
3  chr17  60102  60103  rs1399099657  G  1  A,
4  chr17  60103  60104  rs1363626035  C  1  T,
5  chr17  60149  60150  rs2039333080  C  1  A,
/data/projects/VCF_files/DBSNP/ChrWise_Data/chr2_data.csv


/data/projects/temp/ipykernel_1148936/2691389721.py:4: DtypeWarning: Columns (1,2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(dbsnp_file, header=None, sep= ',', usecols=[0, 1, 2, 3, 4, 5,6])


      0      1      2             3        4  5         6
1  chr2  10006  10007  rs1572047073        C  1        A,
2  chr2  10015  10015  rs1558169263      NaN  1        G,
3  chr2  10016  10023  rs1558169385  CACCCAC  1      CAC,
4  chr2  10018  10021  rs1558169386      CCC  1       CC,
5  chr2  10019  10019  rs1558169388      NaN  2  ACA,ACG,


In [18]:
tsv_files = [f for f in os.listdir("/data/projects/Enhancer/RECOMB_2024/Intersected_data/CoreProm/Chrwise") if f.endswith('.tsv') and "chrM" not in f]

In [19]:
tsv_files

['intersected_DBSNP_transcript_chr2_data.tsv',
 'intersected_DBSNP_transcript_chr13_data.tsv',
 'intersected_DBSNP_transcript_chrY_data.tsv',
 'intersected_DBSNP_transcript_chr19_data.tsv',
 'intersected_DBSNP_transcript_chr21_data.tsv',
 'intersected_DBSNP_transcript_chr7_data.tsv',
 'intersected_DBSNP_transcript_chr12_data.tsv',
 'intersected_DBSNP_transcript_chr9_data.tsv',
 'intersected_DBSNP_transcript_chr6_data.tsv',
 'intersected_DBSNP_transcript_chr3_data.tsv',
 'intersected_DBSNP_transcript_chr4_data.tsv',
 'intersected_DBSNP_transcript_chr11_data.tsv',
 'intersected_DBSNP_transcript_chr15_data.tsv',
 'intersected_DBSNP_transcript_chr22_data.tsv',
 'intersected_DBSNP_transcript_chr18_data.tsv',
 'intersected_DBSNP_transcript_chr16_data.tsv',
 'intersected_DBSNP_transcript_chr10_data.tsv',
 'intersected_DBSNP_transcript_chr1_data.tsv',
 'intersected_DBSNP_transcript_chrX_data.tsv',
 'intersected_DBSNP_transcript_chr14_data.tsv',
 'intersected_DBSNP_transcript_chr17_data.tsv',
 

In [20]:
# Read each .tsv file into a pandas DataFrame and store them in a list
list_of_dfs = []

for file in tsv_files:
    file_path = os.path.join("/data/projects/Enhancer/RECOMB_2024/Intersected_data/CoreProm/Chrwise", file)
    try:
        # Attempt to read the .tsv file
        df = pd.read_csv(file_path, sep='\t', header=None)
        print(df.shape)
        list_of_dfs.append(df)
    except pd.errors.EmptyDataError:
        print(f"Warning: {file_path} is empty and will be skipped.")

(171, 15)
(106, 15)
(325, 15)
(366, 15)
(120, 15)
(238, 15)
(195, 15)
(30, 15)
(440, 15)
(604, 15)
(123, 15)
(139, 15)
(45, 15)
(128, 15)
(38, 15)
(519, 15)
(108, 15)
(286, 15)
(246, 15)
(541, 15)
(35, 15)
(366, 15)


In [21]:
# Concatenate all the DataFrames in the list into a single DataFrame
final_df = pd.concat(list_of_dfs, ignore_index=True) if list_of_dfs else pd.DataFrame()
final_df=final_df.drop(columns=[5, 8])
new_columns = ['chr_name', 'CoreProm_start', 'CoreProm_end', 'Transcript_id', 'Strand', 'Enhancer_start', 'Enhancer_end', 'Variant_start', 'Variant_end', 'RS_ID', 'Reference_Nucleotide', 'No_of_variations', 'Alternative_Nucleotides']
final_df.columns = new_columns
final_df

,chr_name,CoreProm_start,CoreProm_end,Transcript_id,Strand,Enhancer_start,Enhancer_end,Variant_start,Variant_end,RS_ID,Reference_Nucleotide,No_of_variations,Alternative_Nucleotides
0,chr2,178702476,178702565,ENST00000414766.5,-,178702385,178702584,178702478,178702479,rs878862911,G,2,"A,T,"
1,chr2,178702476,178702565,ENST00000414766.5,-,178702385,178702584,178702482,178702483,rs138493804,G,1,"A,"
2,chr2,178702476,178702565,ENST00000414766.5,-,178702385,178702584,178702483,178702484,rs1189632210,T,1,"G,"
3,chr2,178702476,178702565,ENST00000414766.5,-,178702385,178702584,178702485,178702486,rs755668668,A,1,"G,"
4,chr2,178702476,178702565,ENST00000414766.5,-,178702385,178702584,178702487,178702488,rs1052276714,A,1,"C,"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5164,chr5,45226241,45226330,ENST00000671050.1,+,45226090,45226289,45226322,45226323,rs1209221494,T,1,"G,"
5165,chr5,45226241,45226330,ENST00000671050.1,+,45226090,45226289,45226324,45226325,rs1339528998,G,1,"C,"
5166,chr5,45226241,45226330,ENST00000671050.1,+,45226090,45226289,45226325,45226326,rs1743986238,T,1,"C,"
5167,chr5,45226241,45226330,ENST00000671050.1,+,45226090,45226289,45226327,45226328,rs1743986258,C,2,"A,G,"


In [23]:
# Write the final DataFrame to a new .tsv file (only if there's data)
if not final_df.empty:
    output_file_path = "/data/projects/Enhancer/RECOMB_2024/Intersected_data/CoreProm/all_data.tsv"
    final_df.to_csv(output_file_path, sep='\t', index=False)
    print(f"All .tsv files (excluding chrM) concatenated into {output_file_path}")
else:
    print("No data to save after excluding empty files and chrM.")

All .tsv files (excluding chrM) concatenated into /data/projects/Enhancer/RECOMB_2024/Intersected_data/CoreProm/all_data.tsv
